# Estimating Baseline Performance

### What is a baseline model? 

Producing a baseline model is crucial for evaluating your model's performance on any machine learning problem. A baseline model is a basic solution to a machine learning problem that serves as a point of reference for comparing other models to. The baselines model's performance gives us an indication of how much better our models can perform relative to a naive approach. 

Let's say we are building a sentence similarity model where our training set contains pairs of sentences and we want to predict how similiar these sentences are on a scale from 1-5. We could spend months producing a complex machine learning solution to this problem and ultimately get a mean squared error (MSE) of 0.4. But is this result good or bad? There is no way of knowing without comparing it with some baseline performance. For our baseline model, we could predict the mean sentence similarity of sentence pairs in our training set (called the _zero rule_) and get a MSE of 0.2. So our model beats the baseline! If our model's performance was worse than the baseline, this might give us pause to consider using different features, models, evaluation metrics etc. It is crucial that the choice of baseline model be tailored to a data science problem based on buisness goals and the specific modeling task.

### What are good baselines for sentence similarity?

For sentence similarity problems, we have two sub-tasks: 1) First, we need to produce a representation of each sentence in the sentence pair. This representation is called an **embedding**.This embedding allows us to represent a sentence with numbers versus words. Specifically, we're learning an n-dimensional vector of numbers that can represent the given sentence. 2) Second, we need to compute the similarity between these two sentence embeddings.

For producing representations of sentences, there are some common baseline approaches: 
1. Create word embeddings for each word in a sentence
    1. word2vec word embeddings (word2vec)
    2. GLoVe word embeddings (GLoVe)
    
2. Create sentence embeddings
    1. doc2vec document embeddings (doc2vec)
    2. TF-IDF embeddings (TF-IDF)

Then we have to compare our embeddings to calculate sentence similarity:
1. Word Embedding comparison
    1. Cosine Similarity (first requires averaging the word embeddings of all words in each sentence)
    2. Word Mover's Distance

2. Sentence Embedding comparison
    1. Cosine Similarity  
    
The different embedding models and similarity metrics are introduced and explained below.

# Table of Contents
* [Data Loading and Preprocessing](#Data-Loading-and-Preprocessing)
    * [Load STS Benchmark Dataset](#Load-STS-Benchmark-Dataset)
    - [Preprocess / Tokenize](#Data-Preprocessing-/-Tokenization)
    - [Document Frequency Calculation](#Document-Frequency-Calculation)
* [Baseline Models](#Baseline-Models)
    - [Baseline #1: word2vec and cosine similarity](#Baseline-#1:-word2vec-embeddings-+-cosine-similarity-(word2vec-+-cosine))
    - [Baseline #2: word2vec and Word Mover's Distance](#Baseline-#2:-word2vec-embeddings-+-Word-Mover's-Distance-(word2vec-+-WMD))
    - [Baseline #3: GloVe and cosine similarity](#Baseline-#3:-GloVe-embeddings-+-cosine-similarity-(GloVe-+-cosine))
    * [Baseline #4: GloVe and Word Mover's Distance](#Baseline-#4:-GloVe-embeddings-+-Word-Mover's-Distance-(GloVe-+-WMD))
    * [Baseline #5: Doc2vec and cosine similarity](#Baseline-#5:-Doc2vec-embeddings-+-cosine-similarity-(Doc2vec-+-cosine))
    * [Baseline #6: TF-IDF and cosine similarity](#Baseline-#6:-TF-IDF-embeddings-+-cosine-similarity-(TF-IDF-+-cosine))
* [Comparison of Baseline Models](#Comparison-of-Baseline-Models)

# Data Loading and Preprocessing

### Load STS Benchmark Dataset

Here we utilize the [STS Benchmark dataset](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark#STS_benchmark_dataset_and_companion_dataset) which contains a selection of English datasets that were used in Semantic Textual Similarity (STS) tasks 2012-2017. The datasets include text from image captions, news headlines, and user forums. The dataset contains 8,628 sentence pairs with a human-labeled integer representing the sentences' similarity (ranging from 0 for no meaning overlap to 5 meaning equivalence).

In [1]:
import sys
sys.path.append("../../") ## set the environment path
BASE_DATA_PATH = "../../data"

from utils_nlp.dataset.stsbenchmark import STSBenchmark

In [2]:
# Initializing this instance runs the downloader and extractor behind the scenes, then convert to dataframe
stsTrain = STSBenchmark("train", base_data_path=BASE_DATA_PATH).as_dataframe()
stsTest = STSBenchmark("test", base_data_path=BASE_DATA_PATH).as_dataframe()

In [3]:
print(f"Training set has {len(stsTrain)} sentences")
print(f"Testing set has {len(stsTest)} sentences")

Training set has 5749 sentences
Testing set has 1379 sentences


In [4]:
stsTest.head(10)

,score,sentence1,sentence2
0,2.500,A girl is styling her hair.,A girl is brushing her hair.
1,3.600,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.
2,5.000,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.
3,4.200,A man is cutting up a cucumber.,A man is slicing a cucumber.
4,1.500,A man is playing a harp.,A man is playing a keyboard.
5,1.800,A woman is cutting onions.,A woman is cutting tofu.
6,3.500,A man is riding an electric bicycle.,A man is riding a bicycle.
7,2.200,A man is playing the drums.,A man is playing the guitar.
8,2.200,A man is playing guitar.,A lady is playing the guitar.
9,1.714,A man is playing a guitar.,A man is playing a trumpet.


### Data Preprocessing / Tokenization

Our baseline models will expect that each sentence is represented by a list of **tokens**. Tokens are linguistic units like words, punctuation marks, numbers, etc. We'll use the nltk pacakge which is popular for performing tokenization.

from utils_nlp.dataset.preprocess import to_lowercase, to_spacy_tokens

#run this cell if you haven't downloaded en_core_web_sm or hit an error on the below cell
!python -m spacy download en_core_web_sm

df_low = to_lowercase(stsTest)
sts_test = to_spacy_tokens(df_low)

sts_test

It's also common to remove high-frequency words which do not help distinguish one sentence from another, so called **stop words**. For example, "the", "and", "a", etc. are typical stop words although each tokenization package may differ in the words they consider to be stop words. We'll tokenize our corpus without stop words so that we can compare our methods with and without stop words.

In [5]:
import nltk
stop_words = set(nltk.corpus.stopwords.words("english"))

def tokenize(sentence):
    return [t.lower() for t in nltk.word_tokenize(sentence)]

def tokenize_no_stop_words(sentence):
    return [t for t in sentence if t not in stop_words]

In [11]:
sts_test = stsTest
sts_train = stsTrain

sts_train['sentence1_tokens'] = sts_train['sentence1'].apply(tokenize)
sts_train['sentence2_tokens'] = sts_train['sentence2'].apply(tokenize)

sts_train['sentence1_tokens_no_stop'] = sts_train['sentence1_tokens'].apply(tokenize_no_stop_words)
sts_train['sentence2_tokens_no_stop'] = sts_train['sentence2_tokens'].apply(tokenize_no_stop_words)


sts_test['sentence1_tokens'] = sts_test['sentence1'].apply(tokenize)
sts_test['sentence2_tokens'] = sts_test['sentence2'].apply(tokenize)

sts_test['sentence1_tokens_no_stop'] = sts_test['sentence1_tokens'].apply(tokenize_no_stop_words)
sts_test['sentence2_tokens_no_stop'] = sts_test['sentence2_tokens'].apply(tokenize_no_stop_words)

Now each row in our dataframe contains the 2 original sentences as well as a column for each sentence's tokenization with stop words and a column for each sentence's tokenization without stop words.

In [8]:
sts_test.head(5)

,score,sentence1,sentence2,sentence1_tokens,sentence2_tokens,sentence1_tokens_no_stop,sentence2_tokens_no_stop
0,2.5,A girl is styling her hair.,A girl is brushing her hair.,"[a, girl, is, styling, her, hair, .]","[a, girl, is, brushing, her, hair, .]","[girl, styling, hair, .]","[girl, brushing, hair, .]"
1,3.6,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,"[a, group, of, men, play, soccer, on, the, bea...","[a, group, of, boys, are, playing, soccer, on,...","[group, men, play, soccer, beach, .]","[group, boys, playing, soccer, beach, .]"
2,5.0,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,"[one, woman, is, measuring, another, woman, 's...","[a, woman, measures, another, woman, 's, ankle...","[one, woman, measuring, another, woman, 's, an...","[woman, measures, another, woman, 's, ankle, .]"
3,4.2,A man is cutting up a cucumber.,A man is slicing a cucumber.,"[a, man, is, cutting, up, a, cucumber, .]","[a, man, is, slicing, a, cucumber, .]","[man, cutting, cucumber, .]","[man, slicing, cucumber, .]"
4,1.5,A man is playing a harp.,A man is playing a keyboard.,"[a, man, is, playing, a, harp, .]","[a, man, is, playing, a, keyboard, .]","[man, playing, harp, .]","[man, playing, keyboard, .]"


### Document Frequency Calculation

Many of the baseline models we'll explore will require calculation of how frequently a word appears in the sentences of our corpus. In this preprocessing step, we iterate through the sentences in our training set, counting the number of sentences that contain each word. There are other ways to produce this calculation, including pulling larger datasets from the web and calculating the frequency on that data. For example, you could scrap data from Wikipedia or another text heavy site and calculate document frequencies on that corpus. 

In [9]:
def get_document_frequency(df):
    document_frequency_dict = {}
    sentences = df['sentence1'].append(df['sentence2']).apply(tokenize)
    
    for s in sentences:
        for token in set(s):
            if token in document_frequency_dict:
                document_frequency_dict[token] += 1
            else:
                document_frequency_dict[token] = 1
    return document_frequency_dict, len(sentences)

In [12]:
document_frequencies, num_documents = get_document_frequency(sts_train)

In [13]:
num_documents

11498

# Baseline Models

## Baseline #1: word2vec embeddings + cosine similarity (word2vec + cosine)

This baseline constructs word embeddings using word2vec, constructs document embeddings by taking the average of the word embeddings, and then uses cosine similarity to measure sentence similarity.  

### What is Word2Vec?
Word2vec is a predictive model for learning word embeddings from text. Word embeddings are learned such that words that share common contexts in the corpus will be close together in the vector space. There are two different model architectures that can be used to produce word2vec embeddings: continuous bag-of-words (CBOW) or continuous skip-gram. The former uses a window of surrounding words (the "context") to predict the current word and the latter uses the current word to predict the surrounding context words. See this [tutorial](https://www.guru99.com/word-embedding-word2vec.html#3) on word2vec for more detailed background on the model.

For our purposes, we use pretrained word2vec word embeddings. These embeddings were trained on a Google News corpus and provide 300-dimensional embeddings (vectors) for 3 million English words.

### What is Cosine Similarity?

Cosine similarity is a typical similarity metric between vectors. Intuitively it measures the cosine of the angle between any two vectors. With vectors $a$ and $b$, the cosine similarity is: cosine_similarity($a$,$b$) = $\frac{\vec{a} \cdot \vec{b} }{||\vec{a}|| ||\vec{b}||}$


In [14]:
import gensim
import os
PATH_TO_WORD2VEC = os.path.expanduser("GoogleNews-vectors-negative300.bin")
word2vec = gensim.models.KeyedVectors.load_word2vec_format(PATH_TO_WORD2VEC, binary=True)

In [15]:
from collections import Counter
import math
import numpy as np
from scipy.spatial import distance

def average_sentence_embedding(tokens, embedding_model):
     #throw away tokens that are not in embeddings model
    tokens = [i for i in tokens if i in embedding_model]
    
    if len(tokens) == 0:
        return []
    
    #We will weight by TF-IDF:
    #For the TF part: # of times term appears / total terms in sentence
    count = Counter(tokens)
    token_list = list(count)
    term_frequency = [count[i]/len(tokens) for i in token_list]
    
    #Now for the IDF part: LOG(# documents / # documents with term in it)
    inv_doc_frequency = [math.log(num_documents/(document_frequencies.get(i, 0)+1)) for i in count]
    
    #Put the TF-IDF together and produce the weighted average of vector embeddings
    word_embeddings = [embedding_model[token] for token in token_list]
    weights = [term_frequency[i]*inv_doc_frequency[i] for i in range(len(token_list))]
    return list(np.average(word_embeddings, weights = weights, axis = 0))
    
def calculate_cosine_similarity(embedding1, embedding2):
    #distance.cosine calculates cosine DISTANCE, so take 1 - distance to get cosine similarity
    cosine_similarity = 1 - distance.cosine(embedding1, embedding2) 
    return cosine_similarity

In [16]:
def word2vec_average_cosine_similarity(df, embedding_model, stop_words=True):
    predictions = []
    if stop_words:
        tokenized_sentences = zip(df['sentence1_tokens'], df['sentence2_tokens'])
    else:
        tokenized_sentences = zip(df['sentence1_tokens_no_stop'], df['sentence2_tokens_no_stop'])
    
    for (sentence1, sentence2) in tokenized_sentences:
        embedding1 = average_sentence_embedding(sentence1, embedding_model)
        embedding2 = average_sentence_embedding(sentence2, embedding_model)
        if embedding1 == [] or embedding2 == []:
            predictions.append(0)
        else:
            predictions.append(calculate_cosine_similarity(embedding1, embedding2))
    return predictions

## Baseline #2: word2vec embeddings + Word Mover's Distance (word2vec + WMD)

This baseline constructs word embeddings using word2vec and then uses the word mover's distance to measure sentence similarity.  

For an introduction to word2vec, see [Background on Word2Vec](#What-is-Word2Vec?)

### What is Word Mover's Distance (WMD)?
Word Mover's Distance (WMD) is a metric that "adapts the earth mover’s distance to the space of documents: the distance between two texts is given by the total amount of “mass” needed to move the words from one side into the other, multiplied by the distance the words need to move." See this [blog](http://vene.ro/blog/word-movers-distance-in-python.html) for additional information about this similarity measure.

In [17]:
def sentence_embedding(tokens, embedding_model):
     #throw away tokens that are not in embeddings model
    tokens = [i for i in tokens if i in embedding_model]
    
    if len(tokens) == 0:
        return []
    
    count = Counter(tokens)
    token_list = list(count)
    word_embeddings = [list(embedding_model[token]) for token in token_list]
    return word_embeddings


def word2vec_WMD(df, embedding_model, stop_words=True):
    predictions = []
    if stop_words:
        tokenized_sentences = zip(df['sentence1_tokens'], df['sentence2_tokens'])
    else:
        tokenized_sentences = zip(df['sentence1_tokens_no_stop'], df['sentence2_tokens_no_stop'])
    
    for (sentence1, sentence2) in tokenized_sentences:
        #throw away tokens that are not in embeddings model
        tokens1 = [i for i in sentence1 if i in embedding_model]
        tokens2 = [i for i in sentence2 if i in embedding_model]
        if tokens1 == [] or tokens2 == []:
            predictions.append(0)
        else:
            predictions.append(-embedding_model.wmdistance(tokens1, tokens2))
    return predictions

## Baseline #3: GloVe embeddings + cosine similarity (GloVe + cosine)

This baseline constructs word embeddings using GloVE, constructs document embeddings by taking the average of the word embeddings, and then uses cosine similarity to measure sentence similarity.  

### What is GloVe?
GloVe is an unsupervised algorithm for obtaining word embeddings. Training occurs on word-word co-occurance statistics with the objective of learning word embeddings such that the dot product of two word's embeddings is equal to the word's probability of co-occurance. See this [tutorial](https://nlp.stanford.edu/projects/glove/) on GloVe for more detailed background on the model.

For our purposes, we use pretrained GloVe word embeddings (glove.840B.300d.zip which can be downloaded from above link). These embeddings were trained on Common Crawl data and provide 300-dimensional embeddings (vectors) for 2.2 million English words.

For an introduction to the cosine similarity metric, see [Background on Cosine Similarity](#What-is-Cosine-Similarity?)

In [18]:
from gensim.scripts.glove2word2vec import glove2word2vec

#we need to download the GLoVe file and convert it to word2vec format, this takes a bit of time
PATH_TO_GLOVE = os.path.expanduser("glove.840B.300d.txt")
tmp_file = "glove.840B.300d.w2v.txt"
glove2word2vec(PATH_TO_GLOVE, tmp_file)
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)

In [19]:
def glove_average_cosine_similarity(df, embedding_model, stop_words=True):
    predictions = []
    if stop_words:
        tokenized_sentences = zip(df['sentence1_tokens'], df['sentence2_tokens'])
    else:
        tokenized_sentences = zip(df['sentence1_tokens_no_stop'], df['sentence2_tokens_no_stop'])
    
    for (sentence1, sentence2) in tokenized_sentences:
        embedding1 = average_sentence_embedding(sentence1, embedding_model)
        embedding2 = average_sentence_embedding(sentence2, embedding_model)
        if embedding1 == [] or embedding2 == []:
            predictions.append(0)
        else:
            predictions.append(calculate_cosine_similarity(embedding1, embedding2))
    return predictions

## Baseline #4: GloVe embeddings + Word Mover's Distance (GloVe + WMD)

This baseline constructs word embeddings using GloVE and then uses the word mover's distance metric to calculate sentence similarity

For an introduction on GloVe, see [Background on GloVe](#What-is-GloVe?)

For an introduction to WMD, see [Background on Word Mover's Distance](#What-is-Word-Mover's-Distance-(WMD)?)

In [20]:
def glove_WMD(df, embedding_model, stop_words=True):
    predictions = []
    if stop_words:
        tokenized_sentences = zip(df['sentence1_tokens'], df['sentence2_tokens'])
    else:
        tokenized_sentences = zip(df['sentence1_tokens_no_stop'], df['sentence2_tokens_no_stop'])
    
    for (sentence1, sentence2) in tokenized_sentences:
        #throw away tokens that are not in embeddings model
        tokens1 = [i for i in sentence1 if i in embedding_model]
        tokens2 = [i for i in sentence2 if i in embedding_model]
        if tokens1 == [] or tokens2 == []:
            predictions.append(0)
        else:
            predictions.append(-embedding_model.wmdistance(tokens1, tokens2))
    return predictions

## Baseline #5: Doc2vec embeddings + cosine similarity (Doc2vec + cosine)

This baseline constructs document embeddings using doc2vec and then applies cosine similarity to measure each sentence pair's similarity

### What is Doc2Vec?

Doc2vec is an extension of word2vec which produces embeddings of a document. Note that "document" refers to some larger chunk of multiple tokens/words. In our case, our documents will actually be individual setntences. The algorithm not only exploits the idea of context words (like in word2vec), but also incorporates the context of the document. There are again two model architectures that parallel those of word2vec, Paragraph Vectors Distributed Memory (PV-DM) and Paragraph Vectors Distributed Bag-of-Words (PV-DBOW). PV-DM randomly samples consecutive words in a paragraph and predicts a center word by utilizing the context words and the paragraph id. PV-DBOW takes a paragraph id and uses it to predict words in the context. 

See [tutorial #1](https://kanoki.org/2019/03/07/sentence-similarity-in-python-using-doc2vec/) or [tutorial #2](https://gab41.lab41.org/doc2vec-to-assess-semantic-similarity-in-source-code-667acb3e62d7) for more information and an example of using Doc2vec for sentence similarity.

For an introduction to the cosine similarity metric, see [Background on Cosine Similarity](#What-is-Cosine-Similarity?)

In [21]:
#doc2vec requires unique ids for each sentence, so we'll iterate through making a dictionary of sentence to its id
id_dict = {}
n = 0

def assign_id(row):
    global n
    if row not in id_dict:
        id_dict[row] = n
        n += 1
    return id_dict[row]

sts_test['qid1'] = sts_test['sentence1'].apply(assign_id)
sts_test['qid2'] = sts_test['sentence2'].apply(assign_id)

In [54]:
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

def doc2vec_cosine(df, stop_words=True):
    predictions = []
    if stop_words:
        tokenized_sentences = zip(df['sentence1_tokens'], df['sentence2_tokens'])
    else:
        tokenized_sentences = zip(df['sentence1_tokens_no_stop'], df['sentence2_tokens_no_stop'])
        
    labeled_questions=[]
    sentences = list(tokenized_sentences)
    for i in df.index:
        labeled_questions.append(TaggedDocument(sentences[i][0], df[df.index == i].qid1))
        labeled_questions.append(TaggedDocument(sentences[i][1], df[df.index == i].qid2))

    model = Doc2Vec(labeled_questions, dm = 1, min_count=1, window=5, vector_size=500, epochs=30)
        
    for epoch in range(20):
        model.train(labeled_questions,epochs=model.epochs,total_examples=model.corpus_count)
    
    for (sentence1, sentence2) in sentences:
        if len(sentence1) == 0 or len(sentence2) == 0:
            predictions.append(0)
            continue
        score = model.wv.n_similarity(sentence1,sentence2)
        predictions.append(score)
    return predictions

## Baseline #6: TF-IDF embeddings + cosine similarity (TF-IDF + cosine)

This baseline constructs a document embedding based on bag of words with TF-IDF weighting and then applies cosine similarity between the two embeddings in the sentence pair.

### What is TF-IDF?

TF-IDF is a weighting scheme intended to measure how important a word is to the document (or sentence in our case) within the broader corpus (our dataset). The most basic approach for document embeddings is called Bag-of-Words. This method requires first determining the vocabulary across the entire corpus and then, for each document, creating a vector containing the number of times each vocabulary word appeared in the given document. These vectors are obviously very sparse

For an introduction to the cosine similarity metric, see [Background on Cosine Similarity](#What-is-Cosine-Similarity?)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [24]:
def tfidf_cosine_similarity(df, stop_words = True):
    if stop_words:
        tf = TfidfVectorizer(input='content', analyzer='word', min_df = 0, stop_words = 'english', sublinear_tf=True)
    else:
        tf = TfidfVectorizer(input='content', analyzer='word', min_df = 0, stop_words = None, sublinear_tf=True)

    all_sentences = list(df['sentence1'].append(df['sentence2']))
    tfidf_matrix =  tf.fit_transform(all_sentences)
    
    n = 0
    predictions = []
    for (sentence1, sentence2) in zip(df['sentence1'], df['sentence2']):
        sentence1_idx = n
        sentence2_idx = len(sts_test['sentence1'])+n
        sentence1_tfidf = list(tfidf_matrix.getrow(sentence1_idx).toarray()[0])
        sentence2_tfidf = list(tfidf_matrix.getrow(sentence2_idx).toarray()[0])
        if sum(sentence1_tfidf) == 0 or sum(sentence2_tfidf) == 0:
            predictions.append(0)
        else:
            predictions.append(calculate_cosine_similarity(sentence1_tfidf, sentence2_tfidf))
        n += 1
    return predictions

# Comparison of Baseline Models

Our evaluation metric is Pearson correlation which is a measure of the linear correlation between two variables that takes a value in [-1,1] where -1 represents a perfect negative correlation, 1 represents a perfect positive correlation, and 0 represents no correlation. We utilize the Pearson correlation metric as this is the metric that [SentEval](http://nlpprogress.com/english/semantic_textual_similarity.html) , a widely-used evaluation toolkit for evaluation sentence representations, uses for the STS Benchmark dataset.

In [25]:
import scipy
def pearson_correlation(df, prediction):
    pearson_correlation = scipy.stats.pearsonr(prediction, list(df['score']))[0]
    return pearson_correlation

In [56]:
baseline_models = {
    'Word2vec Cosine': word2vec_average_cosine_similarity(sts_test, word2vec, stop_words=False),
    'Word2vec Cosine with Stop Words': word2vec_average_cosine_similarity(sts_test, word2vec, stop_words=True),
    'Word2vec WMD' : word2vec_WMD(sts_test, word2vec, stop_words=False),
    'Word2vec WMD with Stop Words' : word2vec_WMD(sts_test, word2vec, stop_words=True),
    'GLoVe Cosine': glove_average_cosine_similarity(sts_test, glove, stop_words=False),
    'GLoVe Cosine with Stop Words': glove_average_cosine_similarity(sts_test, glove, stop_words=True),
    'GLoVe WMD' : glove_WMD(sts_test, glove, stop_words=False),
    'GLoVe WMD with Stop Words' : glove_WMD(sts_test, glove, stop_words=True),
    'Doc2vec Cosine': doc2vec_cosine(sts_test, stop_words=False),
    'Doc2vec Cosine with Stop Words': doc2vec_cosine(sts_test, stop_words=True),
    'TF-IDF Cosine': tfidf_cosine_similarity(sts_test, stop_words = False),
    'TF-IDF Cosine with Stop Words': tfidf_cosine_similarity(sts_test, stop_words = True),
}

In [57]:
for model in baseline_models:
    print(model, pearson_correlation(sts_test, baseline_models[model]))

Word2vec Cosine 0.6252265308481969
Word2vec Cosine with Stop Words 0.6425244884199678
Word2vec WMD 0.6434252846415083
Word2vec WMD with Stop Words 0.5578145579252304
GLoVe Cosine 0.6224951988473979
GLoVe Cosine with Stop Words 0.5540268255219093
GLoVe WMD 0.6198138203783634
GLoVe WMD with Stop Words 0.4795588645934349
Doc2vec Cosine 0.5174164747271393
Doc2vec Cosine with Stop Words 0.3634126862348207
TF-IDF Cosine 0.7034695168223283
TF-IDF Cosine with Stop Words 0.6683811410442564


The results show that TF-IDF bag-of-words document embeddings combined with the cosine similarity performs the best, with a Pearson correlation of 0.7034. 